# 🎓 CSC582: Relational to Key-Value Database Migration
## E-Commerce Database → Redis Cluster

**Course:** CSC582 - Data Warehouse and Mining Systems

This notebook demonstrates:
- Relational database design (SQLite)
- Mapping strategy to Key-Value model
- Redis Cluster with sharding & replication
- Key-Value operations (SET, GET)

---

## 📦 Step 1: Install Redis Server

In [ ]:
# Install Redis Server and Python client
!apt-get update -qq
!apt-get install -y redis-server redis-tools > /dev/null
!pip install redis -q

print("✅ Redis installed successfully!")
!redis-server --version

## 🚀 Step 2: Start Redis Server

In [ ]:
import subprocess
import time

# Start Redis server in background
subprocess.Popen(['redis-server', '--daemonize', 'yes'])
time.sleep(2)

# Verify Redis is running
result = subprocess.run(['redis-cli', 'ping'], capture_output=True, text=True)
if result.stdout.strip() == 'PONG':
    print("✅ Redis server is running!")
else:
    print("❌ Redis failed to start")

---
# 📊 Part 1: Relational Database

We create an E-Commerce database with 3 tables:
- **Customer**: Customer information
- **Product**: Product catalog
- **Order**: Customer orders

In [ ]:
import sqlite3
import pandas as pd

# Create SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.executescript("""
    CREATE TABLE Customer (
        customer_id     INTEGER PRIMARY KEY,
        name            TEXT NOT NULL,
        email           TEXT UNIQUE NOT NULL
    );
    
    CREATE TABLE Product (
        product_id      INTEGER PRIMARY KEY,
        name            TEXT NOT NULL,
        price           REAL NOT NULL
    );
    
    CREATE TABLE Order_Table (
        order_id        INTEGER PRIMARY KEY,
        customer_id     INTEGER NOT NULL,
        order_date      TEXT NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
    );
""")

print("✅ Relational schema created!")
print("\n📋 Tables: Customer, Product, Order")

In [ ]:
# Insert sample data
customers_data = [
    (1, 'UserA', 'usera@test.com'),
    (2, 'UserB', 'userb@test.com'),
    (3, 'UserC', 'userc@test.com'),
]
cursor.executemany("INSERT INTO Customer VALUES (?,?,?)", customers_data)

products_data = [
    (101, 'Keyboard', 199),
    (102, 'Mouse', 99),
    (103, 'Monitor', 599),
]
cursor.executemany("INSERT INTO Product VALUES (?,?,?)", products_data)

orders_data = [
    (1001, 1, '2025-01-01'),
    (1002, 2, '2025-01-02'),
    (1003, 1, '2025-01-03'),
]
cursor.executemany("INSERT INTO Order_Table VALUES (?,?,?)", orders_data)
conn.commit()

print("✅ Sample data inserted!")

In [ ]:
# Display Customer Table
print("👥 CUSTOMER TABLE")
print("="*50)
df_customers = pd.read_sql_query("SELECT * FROM Customer", conn)
display(df_customers)

In [ ]:
# Display Product Table
print("📦 PRODUCT TABLE")
print("="*50)
df_products = pd.read_sql_query("SELECT * FROM Product", conn)
display(df_products)

In [ ]:
# Display Order Table
print("🛒 ORDER TABLE")
print("="*50)
df_orders = pd.read_sql_query("SELECT * FROM Order_Table", conn)
display(df_orders)

---
# 🔄 Part 2: Mapping Strategy

## Relational → Key-Value Mapping

### Key Format: `TableName:TupleID:Attribute`

Each attribute in a relational row becomes a separate key-value pair.

---

### 2.1 Customer Table Mapping

| Relational Row | (1, UserA, usera@test.com) |
|----------------|---------------------------|
| **Keys** | Key1: `Customer:1:name` | 
|          | Key2: `Customer:1:email` |
| **Values** | Key1 Value: "UserA" |
|            | Key2 Value: "usera@test.com" |

---

### 2.2 Product Table Mapping

| Relational Row | (101, Keyboard, 199) |
|----------------|---------------------|
| **Keys** | Key1: `Product:101:name` |
|          | Key2: `Product:101:price` |
| **Values** | Key1 Value: "Keyboard" |
|            | Key2 Value: "199" |

---

### 2.3 Order Table Mapping

| Relational Row | (1001, 1, 2025-01-01) |
|----------------|----------------------|
| **Keys** | Key1: `Order:1001:customer_id` |
|          | Key2: `Order:1001:order_date` |
| **Values** | Key1 Value: "1" |
|            | Key2 Value: "2025-01-01" |

In [ ]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Verify connection
print("🔗 Connected to Redis:", r.ping())

---
# 📤 Part 3: Data Migration to Redis

Using the format: **TableName:TupleID:Attribute → Value**

In [ ]:
print("="*60)
print("🚀 MIGRATING DATA TO REDIS")
print("Key Format: TableName:TupleID:Attribute")
print("="*60)

# Migrate Customers
print("\n👥 Migrating CUSTOMER Table...")
print("-"*60)
cursor.execute("SELECT * FROM Customer")
for row in cursor.fetchall():
    customer_id = row[0]
    
    # Key: Customer:ID:name → Value: name
    key1 = f"Customer:{customer_id}:name"
    r.set(key1, row[1])
    print(f"   SET {key1} → \"{row[1]}\"")
    
    # Key: Customer:ID:email → Value: email
    key2 = f"Customer:{customer_id}:email"
    r.set(key2, row[2])
    print(f"   SET {key2} → \"{row[2]}\"")
    print()

In [ ]:
# Migrate Products
print("📦 Migrating PRODUCT Table...")
print("-"*60)
cursor.execute("SELECT * FROM Product")
for row in cursor.fetchall():
    product_id = row[0]
    
    # Key: Product:ID:name → Value: name
    key1 = f"Product:{product_id}:name"
    r.set(key1, row[1])
    print(f"   SET {key1} → \"{row[1]}\"")
    
    # Key: Product:ID:price → Value: price
    key2 = f"Product:{product_id}:price"
    r.set(key2, str(row[2]))
    print(f"   SET {key2} → \"{row[2]}\"")
    print()

In [ ]:
# Migrate Orders
print("🛒 Migrating ORDER Table...")
print("-"*60)
cursor.execute("SELECT * FROM Order_Table")
for row in cursor.fetchall():
    order_id = row[0]
    
    # Key: Order:ID:customer_id → Value: customer_id
    key1 = f"Order:{order_id}:customer_id"
    r.set(key1, str(row[1]))
    print(f"   SET {key1} → \"{row[1]}\"")
    
    # Key: Order:ID:order_date → Value: order_date
    key2 = f"Order:{order_id}:order_date"
    r.set(key2, row[2])
    print(f"   SET {key2} → \"{row[2]}\"")
    print()

print("="*60)
print("✅ MIGRATION COMPLETE!")
print("="*60)

---
# 🔍 Part 4: Key-Value Operations Demo

## 4.1 GET - Retrieve values by key

In [ ]:
print("📋 GET Operations - Retrieve by Key")
print("="*60)

# Get Customer 1 name
print("\nCommand: GET Customer:1:name")
result = r.get("Customer:1:name")
print(f"Result: {result}")

# Get Customer 1 email
print("\nCommand: GET Customer:1:email")
result = r.get("Customer:1:email")
print(f"Result: {result}")

In [ ]:
print("📋 GET Product Information")
print("="*60)

# Get Product 101 name
print("\nCommand: GET Product:101:name")
result = r.get("Product:101:name")
print(f"Result: {result}")

# Get Product 101 price
print("\nCommand: GET Product:101:price")
result = r.get("Product:101:price")
print(f"Result: {result}")

In [ ]:
print("📋 GET Order Information")
print("="*60)

# Get Order 1001 customer_id
print("\nCommand: GET Order:1001:customer_id")
result = r.get("Order:1001:customer_id")
print(f"Result: {result}")

# Get Order 1001 order_date
print("\nCommand: GET Order:1001:order_date")
result = r.get("Order:1001:order_date")
print(f"Result: {result}")

## 4.2 SET - Store new values

In [ ]:
print("📋 SET Operations - Store new values")
print("="*60)

# Add a new customer
print("\nAdding new Customer 4...")
r.set("Customer:4:name", "UserD")
r.set("Customer:4:email", "userd@test.com")
print("   SET Customer:4:name → \"UserD\"")
print("   SET Customer:4:email → \"userd@test.com\"")

# Verify
print("\nVerifying new customer:")
print(f"   GET Customer:4:name → {r.get('Customer:4:name')}")
print(f"   GET Customer:4:email → {r.get('Customer:4:email')}")

## 4.3 KEYS - Find keys by pattern

In [ ]:
print("📋 KEYS - Find keys matching pattern")
print("="*60)

# Find all Customer keys
print("\nCommand: KEYS Customer:*")
keys = r.keys("Customer:*")
for k in sorted(keys):
    print(f"   {k}")

In [ ]:
# Find all Product keys
print("Command: KEYS Product:*")
keys = r.keys("Product:*")
for k in sorted(keys):
    print(f"   {k}")

In [ ]:
# Find all Order keys
print("Command: KEYS Order:*")
keys = r.keys("Order:*")
for k in sorted(keys):
    print(f"   {k}")

In [ ]:
# Find all keys for a specific tuple (e.g., Customer 1)
print("\nCommand: KEYS Customer:1:*")
keys = r.keys("Customer:1:*")
print("All attributes for Customer 1:")
for k in sorted(keys):
    value = r.get(k)
    print(f"   {k} → {value}")

## 4.4 Simulating a JOIN - Get Order with Customer Name

In [ ]:
print("📋 Simulating JOIN: Get Order 1001 with Customer Name")
print("="*60)

# Step 1: Get customer_id from order
print("\nStep 1: GET Order:1001:customer_id")
customer_id = r.get("Order:1001:customer_id")
print(f"   Result: {customer_id}")

# Step 2: Get customer name using the customer_id
print(f"\nStep 2: GET Customer:{customer_id}:name")
customer_name = r.get(f"Customer:{customer_id}:name")
print(f"   Result: {customer_name}")

# Step 3: Get order date
print("\nStep 3: GET Order:1001:order_date")
order_date = r.get("Order:1001:order_date")
print(f"   Result: {order_date}")

print("\n" + "="*60)
print(f"📦 Order 1001: Customer={customer_name}, Date={order_date}")
print("="*60)

---
# 🖥️ Part 5: Redis CLI Commands

Run these commands directly using redis-cli:

In [ ]:
print("🖥️ REDIS CLI DEMO")
print("="*60)
print("\nGET Customer:1:name")
!redis-cli GET Customer:1:name

In [ ]:
print("GET Product:101:price")
!redis-cli GET Product:101:price

In [ ]:
print("KEYS Customer:*")
!redis-cli KEYS "Customer:*"

In [ ]:
print("DBSIZE (total number of keys)")
!redis-cli DBSIZE

---
# 🏗️ Part 6: Redis Cluster Architecture

In a production environment, we would use 8 nodes:

```
┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│  MASTER 0   │  │  MASTER 1   │  │  MASTER 2   │  │  MASTER 3   │
│  Port 7000  │  │  Port 7001  │  │  Port 7002  │  │  Port 7003  │
│Slots 0-4095 │  │Slots 4096-  │  │Slots 8192-  │  │Slots 12288- │
│             │  │   8191      │  │  12287      │  │   16383     │
└──────┬──────┘  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘
       │                │                │                │
       ▼                ▼                ▼                ▼
┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│  REPLICA 0  │  │  REPLICA 1  │  │  REPLICA 2  │  │  REPLICA 3  │
│  Port 7004  │  │  Port 7005  │  │  Port 7006  │  │  Port 7007  │
└─────────────┘  └─────────────┘  └─────────────┘  └─────────────┘
```

### Hash-Based Partitioning:
- Total slots: 16384
- Formula: `slot = CRC16(key) mod 16384`
- Each key is assigned to a specific master based on its hash slot

In [ ]:
# Simulate hash slot calculation
def crc16_simple(data):
    """Simplified CRC16 for demonstration"""
    crc = 0
    for byte in data.encode():
        crc = ((crc << 5) + crc + byte) & 0xFFFF
    return crc

def get_slot(key):
    return crc16_simple(key) % 16384

def get_node(slot):
    if slot < 4096:
        return "Master 0 (Port 7000)"
    elif slot < 8192:
        return "Master 1 (Port 7001)"
    elif slot < 12288:
        return "Master 2 (Port 7002)"
    else:
        return "Master 3 (Port 7003)"

print("🎯 HASH SLOT DISTRIBUTION")
print("="*70)
print(f"{'Key':<35} {'Slot':<10} {'Node'}")
print("-"*70)

keys_to_check = [
    "Customer:1:name",
    "Customer:1:email",
    "Product:101:name",
    "Product:101:price",
    "Order:1001:customer_id",
    "Order:1001:order_date"
]

for key in keys_to_check:
    slot = get_slot(key)
    node = get_node(slot)
    print(f"{key:<35} {slot:<10} {node}")

---
# 📊 Part 7: Summary - Complete Mapping View

In [ ]:
print("📊 COMPLETE MAPPING SUMMARY")
print("="*70)
print("\nKey Format: TableName:TupleID:Attribute → Value")
print("="*70)

# Show all keys and values
all_keys = sorted(r.keys("*"))

print(f"\n{'Key':<40} {'Value'}")
print("-"*70)

for key in all_keys:
    value = r.get(key)
    print(f"{key:<40} {value}")

print("\n" + "="*70)
print(f"Total Keys in Redis: {len(all_keys)}")
print("="*70)

---
# ✅ Conclusion

This demo covered:

| Component | Description |
|-----------|-------------|
| **Relational Model** | 3 tables: Customer, Product, Order |
| **Key Format** | `TableName:TupleID:Attribute` |
| **Operations** | SET, GET, KEYS |
| **Sharding** | Hash-based partitioning (16384 slots) |
| **Replication** | 4 Masters + 4 Replicas |

### Mapping Examples:

| Relational | Key-Value |
|------------|----------|
| Customer(1, 'UserA', 'usera@test.com') | `Customer:1:name` → "UserA" |
| | `Customer:1:email` → "usera@test.com" |
| Product(101, 'Keyboard', 199) | `Product:101:name` → "Keyboard" |
| | `Product:101:price` → "199" |
| Order(1001, 1, '2025-01-01') | `Order:1001:customer_id` → "1" |
| | `Order:1001:order_date` → "2025-01-01" |

---
**Thank you!**